In [ ]:
from requests import get, post, put
import os
import pandas as pd
from sensitive import *

In [ ]:
auth = f"Bearer {AUTH_TOKEN}"
headers = {"Accept": "application/json",
           "Content-Type": "application/json", "Authorization": auth}


In [31]:
artists = {}
songs = []


def get_total_song_count():
    playlist_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}', headers=headers)
    return playlist_res.json()["tracks"]["total"]


def add_genres(list, genres):
    for genre in genres:
        if genre not in list:
            list.append(genre)

    return list


def get_song_genres(song):
    genres = []
    for artist in song["track"]["artists"]:
        id = artist["id"]
        if id not in artists:
            artist_genres_res = get(
                f'https://api.spotify.com/v1/artists/{id}', headers=headers)
            try:
                artist_genres = artist_genres_res.json()["genres"]
            except Exception:
                print(artist_genres_res.json())
            genres = add_genres(genres, artist_genres)
            artists[artist["name"]] = artist_genres
        else:
            genres = add_genres(genres, artists[id])

    return genres


def song_data(song):
    return song["track"]['id'], song["track"]['name'], song["track"]['popularity'], [artist["name"] for artist in song["track"]["artists"]]


In [32]:
total_song_count = get_total_song_count()

In [33]:


all_genres = []
for offset in range(0, total_song_count, 100):
    all_genres_res = get(
        f'https://api.spotify.com/v1/playlists/{PLAYLIST_ID}/tracks?limit=100&offset={offset}', headers=headers)
    for song in all_genres_res.json()["items"]:
        (id, name, popularity, artist), song_genres = song_data(
            song), get_song_genres(song)
        songs.append({"id": id, "name": name, "artists": artist,
                     "popularity": popularity, "genres": song_genres})
        all_genres = add_genres(all_genres, song_genres)


In [34]:
songs = songs[-total_song_count:]

In [35]:
playlist = pd.DataFrame(data=songs, columns=[
                        'id', 'name', 'artists', 'popularity', 'genres'])


In [36]:
playlist


,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"[Justin Bieber, Dominic Fike]",57,"[canadian pop, pop, alternative pop rock]"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"[Kina, Snøw]",83,"[sad lo-fi, sad rap]"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,[Alessia Cara],77,"[canadian contemporary r&b, canadian pop, danc..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"[Teqkoi, Snøw]",69,"[emo rap, lo-fi chill, sad lo-fi, sad rap]"
4,3xMTM729pJSwUv55WwWcm2,change,[One Hope],44,[sad lo-fi]
...,...,...,...,...,...
980,0R7Hkh6bkomGA6kB62BW1Z,i love you but you make me cry,[Elthia],45,[gen z singer-songwriter]
981,6OVpQb3zarqu0vfQav0Wwx,thenineteenseventyfive,[push baby],53,[uk pop]
982,0dOY7HTtl9OhlouW5ToZNC,"I'm Leaving, Sorry For Your Loss",[iamnotshane],49,[]
983,7byGwLkiIzJdHOZKZMS8nd,Something Super Sweet,[Rory Webley],65,"[alt z, social media pop]"


In [37]:
playlist.to_csv('playlist.csv', header=True, index=None)

In [38]:
all_artists = list(artists.keys())
playlist = pd.read_csv('playlist.csv')
playlist

,id,name,artists,popularity,genres
0,6Z2pe32hFnUd2x5NgO5KFc,Die For You (feat. Dominic Fike),"['Justin Bieber', 'Dominic Fike']",57,"['canadian pop', 'pop', 'alternative pop rock']"
1,4ZLzoOkj0MPWrTLvooIuaa,Get You The Moon (feat. Snøw),"['Kina', 'Snøw']",83,"['sad lo-fi', 'sad rap']"
2,4WzhjxvLP95y7AMDy0Atwb,Out Of Love,['Alessia Cara'],77,"['canadian contemporary r&b', 'canadian pop', ..."
3,4P1L6AniTDJzANaubzWGYs,I'm Not Enough and I'm Sorry,"['Teqkoi', 'Snøw']",69,"['emo rap', 'lo-fi chill', 'sad lo-fi', 'sad r..."
4,3xMTM729pJSwUv55WwWcm2,change,['One Hope'],44,['sad lo-fi']
...,...,...,...,...,...
980,0R7Hkh6bkomGA6kB62BW1Z,i love you but you make me cry,['Elthia'],45,['gen z singer-songwriter']
981,6OVpQb3zarqu0vfQav0Wwx,thenineteenseventyfive,['push baby'],53,['uk pop']
982,0dOY7HTtl9OhlouW5ToZNC,"I'm Leaving, Sorry For Your Loss",['iamnotshane'],49,[]
983,7byGwLkiIzJdHOZKZMS8nd,Something Super Sweet,['Rory Webley'],65,"['alt z', 'social media pop']"


In [39]:
def genres_indexed(genres: list[str]) -> list[int]:
    indexed = []
    for all_genres_x in all_genres:

        continue_outer = False
        for genre in genres:
            index = all_genres_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


def artists_indexed(artists: list[str]) -> list[int]:
    indexed = []
    for all_artists_x in all_artists:

        continue_outer = False
        for genre in artists:
            index = all_artists_x == genre
            if index:
                continue_outer = True
                indexed.append(int(True))
                break

        if continue_outer:
            continue

        indexed.append(int(False))

    return indexed


In [40]:
playlist["genres_indexed"] = [genres_indexed(
    eval(playlist["genres"][x])) for x in range(len(playlist["genres"]))]


In [41]:
playlist.columns = playlist.columns.map(lambda x: x.strip())

In [42]:
playlist["artists_indexed"] = [artists_indexed(
    eval(playlist["artists"][x])) for x in range(len(playlist["artists"]))]


In [43]:
playlist.columns


Index(['id', 'name', 'artists', 'popularity', 'genres', 'genres_indexed',
       'artists_indexed'],
      dtype='object')

## KNN


In [44]:
def knn_prepared_data(df):
    df = df[['id', 'name', 'genres', 'artists',
             'popularity', 'genres_indexed', 'artists_indexed']]
    list = []
    for index in range(len(df['id'])):
        list.append({'id': df['id'][index], 'name': df['name'][index], 'genres': df['genres'][index], 'artists': df['artists'][index],
                    'popularity': df['popularity'][index], 'genres_indexed': df['genres_indexed'][index], 'artists_indexed': df['artists_indexed'][index]})

    return list


In [45]:
song_dict = knn_prepared_data(playlist)

In [46]:
import operator


def getNeighbors(song, K):
    distances = []
    for song_index, song_value in enumerate(song_dict):
        if (song_index != song):
            dist = ComputeDistance(song_dict[song], song_value)
            distances.append((song_index, dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors


In [47]:
def list_distance(a, b):
    distance = 0
    for item_a, item_b in list(zip(a, b)):
        distance += 0 if item_a == item_b else 0.2 if (len(a) < 400) else 0.1

    return distance

In [93]:
def ComputeDistance(a, b):
    genres_distance = list_distance(a['genres_indexed'], b['genres_indexed'])
    artists_distance = list_distance(a['artists_indexed'], b['artists_indexed'])
    popularity_distance = abs(a['popularity'] - b['popularity'])
    
    return genres_distance * 1.3 + artists_distance * 0.2 + popularity_distance * 0.004


## Song options

### Choose one of these to get the 50 closest songs to it


In [102]:
# Set the song name below
top_5 = get('https://api.spotify.com/v1/me/top/tracks?time_range=medium_term&limit=5', headers=headers).json()
song_name = ''
top_5_names = list(map(lambda song: song['name'],top_5['items']))
for i in top_5_names:
    if i in list(playlist['name']):
        song_name = i
        break


In [108]:

K = 99
item = playlist[[playlist['name'][x] ==
                 song_name for x in range(len(playlist['name']))]]
index = item.index[0]
neighbors = getNeighbors(index, K)
print(song_dict[index]['name'], song_dict[index]['artists'],
      song_dict[index]['genres'], song_dict[index]['popularity'])
for neighbor, index in zip(neighbors, range(1, K + 1)):
    print(
        f'{index}. {song_dict[neighbor]["name"]} \t {song_dict[neighbor]["artists"]} \t {song_dict[neighbor]["genres"]} \t {song_dict[neighbor]["popularity"]}')


The Story of the Paper Boy ['Powfu'] ['sad rap'] 51
1. when things were awkward 	 ['Powfu'] 	 ['sad rap'] 	 50
2. Different Class, Same Boat 	 ['Powfu'] 	 ['sad rap'] 	 53
3. Days We Had 	 ['Powfu'] 	 ['sad rap'] 	 54
4. I Know You're Not Happy 	 ['Powfu'] 	 ['sad rap'] 	 54
5. Her Ocean Eyes 	 ['Powfu'] 	 ['sad rap'] 	 48
6. Breakfast With the Moon 	 ['Powfu'] 	 ['sad rap'] 	 55
7. clash of the titans 	 ['Powfu'] 	 ['sad rap'] 	 46
8. Hear my thoughts 	 ['Powfu'] 	 ['sad rap'] 	 56
9. no promises (feat. Scapegoat) 	 ['Powfu', 'Scapegoat'] 	 ['sad rap'] 	 55
10. Douchebags 	 ['Powfu'] 	 ['sad rap'] 	 41
11. Haunted 	 ['Zimm'] 	 ['sad rap'] 	 51
12. im used to it 	 ['Powfu'] 	 ['sad rap'] 	 62
13. step into my life 	 ['Powfu', 'sleep.ing'] 	 ['sad rap'] 	 59
14. the fire in your eyes keeps me warm 	 ['Powfu', 'sleep.ing', 'Arvnd'] 	 ['sad rap'] 	 55
15. every time you wanna see a sunlight 	 ['Wikw'] 	 ['sad rap'] 	 44
16. i heard heaven is a nice place 	 ['Carter Powers'] 	 ['sad rap'] 

In [109]:
import json
data = {"name": f"Recent-ish Favorites", "description": f"Songs related to your medium term top 5", "public": False}
playlist_creation = post(f'https://api.spotify.com/v1/users/{USER_ID}/playlists', headers=headers, data=json.dumps(data))
new_id = playlist_creation.json()['id']

In [110]:
item = playlist[[playlist['name'][x] ==
                 song_name for x in range(len(playlist['name']))]]
index = item.index[0]
song_uris = f'spotify:track:{song_dict[index]["id"]}'
for neighbor in neighbors:
    song_uris += f',spotify:track:{song_dict[neighbor]["id"]}'

add_songs_req = post(f'https://api.spotify.com/v1/playlists/{new_id}/tracks?uris={song_uris}', headers=headers, data=json.dumps({}))
add_songs_req.json()

{'snapshot_id': 'MyxlN2NiNWViY2NjMWYyMTUzMzIwYjZkOTcwYWE5ZWRhZjMzNWYxZTIz'}